# Notebook to estimate/interpolate original coordinates of RICS images

In [65]:
import numpy as np
from  glob import glob
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
# from math import radians, sin, cos, sqrt, atan2
import math
import os
from shapely.geometry import Point
from scipy import interpolate

In [ ]:
input_folder=r"C:\Users\lliu\FrontierSI\Projects - 127 Residential Dwelling Floor Height\4 Executing\GA_data_documentation\RICS_DB_geojson"
out_folder=r"C:\Users\lliu\Desktop\FrontierSI\projects\GA_floor_height\GA-floor-height\output\Wagga\RICS"
time_lag_seconds=1.5
frame_per_sec = 0.25
gap_threshold = 30
# Loop through all files in the folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".geojson"):
        # File reading and writing
        input_file = os.path.join(input_folder, file_name)
        # Read input GeoJSON
        gdf = gpd.read_file(input_file)
        gdf["time"] = gdf["time"].astype('float32')
        gdf= gdf.sort_values(by='frame')
        unique_times=np.unique(gdf["time"])
        gdf_interpolated=[]

        for time_int in unique_times:
            gdf_slice=gdf[gdf["time"]==time_int]
            n_duplicates = len(gdf_slice)
            if n_duplicates>1:
                gdf_slice = gdf_slice.sort_values('frame').reset_index()
                for i in range(1,n_duplicates):
                    gdf_slice.loc[i,"time"]=gdf_slice.loc[i,"time"]+(gdf_slice.loc[i,"frame"]-gdf_slice.loc[0,"frame"])*frame_per_sec
            gdf_interpolated.append(gdf_slice)

        gdf_interpolated=pd.concat(gdf_interpolated, ignore_index=True)
        gdf_interpolated["time"] = gdf_interpolated["time"] + time_lag_seconds   
        gdf_interpolated=gdf_interpolated.sort_values(by='frame')
        # f_lon = interpolate.interp1d(gdf["time"], gdf.centroid.x, fill_value='extrapolate')
        # f_lat = interpolate.interp1d(gdf["time"], gdf.centroid.y, fill_value='extrapolate')

        # Identify indices where gaps occur
        gaps = np.diff(gdf_interpolated["time"]) > gap_threshold
        segment_indices = np.where(gaps)[0] + 1
        # Split data into segments
        segments_x = np.split(gdf["time"], segment_indices)
        segments_y_lon = np.split(gdf.centroid.x, segment_indices)
        segments_y_lat = np.split(gdf.centroid.y, segment_indices)
        segments_x_new_times=np.split(gdf_interpolated["time"], segment_indices)

        # Interpolation for each segment
        interpolated_segments_lon = []
        interpolated_segments_lat = []
        interpolated_segments_times=[]
        for seg_x, seg_y_lon, seg_y_lat, seg_x_new_time in zip(segments_x, segments_y_lon, segments_y_lat, segments_x_new_times):
            seg_x,ind=np.unique(seg_x,return_index=True)
            # Create interpolation function for the segment
            f_lon = interpolate.interp1d(seg_x, seg_y_lon.to_numpy().ravel()[ind], kind='linear', fill_value="extrapolate")
            f_lat = interpolate.interp1d(seg_x, seg_y_lat.to_numpy().ravel()[ind], kind='linear', fill_value="extrapolate")
            seg_interp_lon = f_lon(seg_x_new_time)
            seg_interp_lat = f_lat(seg_x_new_time)
            interpolated_segments_lon.append(pd.DataFrame(data=seg_interp_lon,columns=['lon']))
            interpolated_segments_lat.append(pd.DataFrame(data=seg_interp_lat,columns=['lat']))
            interpolated_segments_times.append(pd.DataFrame(data=seg_x_new_time,columns=['time']))

        interpolated_segments_lon=pd.concat(interpolated_segments_lon, ignore_index=True)
        interpolated_segments_lat=pd.concat(interpolated_segments_lat, ignore_index=True)
        interpolated_segments_times=pd.concat(interpolated_segments_times, ignore_index=True)
        merged_interp=pd.concat([interpolated_segments_lon,interpolated_segments_lat,interpolated_segments_times], axis=1)

        gdf_interpolated=pd.merge(gdf_interpolated, merged_interp, on='time', how='right')
        # interpolated_lon = f_lon(gdf_interpolated["time"])
        # interpolated_lat = f_lat(gdf_interpolated["time"])
        # gdf_interpolated["lon"] = interpolated_lon
        # gdf_interpolated["lat"] = interpolated_lat
        gdf_interpolated['geometry'] = gdf_interpolated.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
        gdf_interpolated=gpd.GeoDataFrame(data=gdf_interpolated)
        #######################################################################
        # Save to a new GeoJSON file
        out_file=os.path.join(out_folder,os.path.basename(input_file).replace('.geojson','_interpolated.geojson'))
        gdf_interpolated.to_file(out_file, driver="GeoJSON")

c:\Users\lliu\miniconda3\envs\floorheight\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\lliu\AppData\Local\Temp\ipykernel_24460\1253461249.py:51: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  segments_y_lon = np.split(gdf.centroid.x, segment_indices)
C:\Users\lliu\AppData\Local\Temp\ipykernel_24460\1253461249.py:52: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  segments_y_lat = np.split(gdf.centroid.y, segment_indices)
c:\Users\lliu\miniconda3\envs\floorheight\lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: invalid valu